In [1]:
import requests

In [12]:
def main():
    for article_url in article_urls:
        send_line_notify('ニュース', article_url)

def send_line_notify(notification_message, url):
    """
    Lineに通知する
    """
    line_notify_token = 'LMr5ZeQuAsAz6GMxaMJyzLuhGBO3VvVIFxTwfSLUvz0'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': url}
    requests.post(line_notify_api, headers = headers, data = data)

if __name__ == "__main__":
    main()

In [3]:
from bs4 import BeautifulSoup
import requests
import time

def scraping_news():
    url = 'https://news.yahoo.co.jp/'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    elements = soup.select('.sc-fKgJPI.fFnFHR')
    
    topics_list = []
    for element in elements:
        url_href = element.a['href']
        topics_list.append(url_href)
        time.sleep(1)
    
    topics_list.pop(-1)
    
    topics_urls_list = []
    for topic in topics_list:
        topics_urls_list.append(url + topic)
    
    articles_dict = []
    
    for url in topics_urls_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        
        elements = soup.select('.sc-fXazdy.UjHkE')
        elements = elements[:5]
    
        news_urls = []
        news_titles = []
        for element in elements:
            url = element.a['href']
            title = element.text

            news_urls.append(url)
            news_titles.append(title)
            
            time.sleep(1)
    
        news_text = []
        for news_url in news_urls:
            r = requests.get(news_url)
            soup = BeautifulSoup(r.text, "html.parser")
            
            elements = soup.select('.sc-dYXZXt.jzZUwL.yjSlinkDirectlink.highLightSearchTarget')
            text = elements[0].text
            
            news_text.append(text)
    
            time.sleep(1)
            
        articles = []
        for url, title, text in zip(news_urls, news_titles, news_text):
            article = {
                "title" : title,
                "URL" : url,
                "text" : text
            }
            articles.append(article)
        articles_dict.append(articles)

    return articles_dict


In [4]:
articles_dict = scraping_news()

In [5]:
for i in range(len(articles_dict)):
    for j in range(len(articles_dict[0])):
        articles_dict[i][j]['title'] = articles_dict[i][j]['title'].replace('\u3000', '') 
        articles_dict[i][j]['title'] = articles_dict[i][j]['title'].lstrip('0123456789')
        articles_dict[i][j]['text'] = articles_dict[i][j]['text'].replace('\u3000', '')
        articles_dict[i][j]['text'] = articles_dict[i][j]['text'].replace('\n', '')

In [7]:
article_dict = articles_dict[0]
article_urls = []
for i in article_dict:
    article_urls.append(i['URL'])

In [17]:
article_urls

['https://news.yahoo.co.jp/articles/3645384478280614935345122a6ef4ce1a18b2c0',
 'https://news.yahoo.co.jp/articles/158fc26d76f503af49a473ec697b956565b16d23',
 'https://news.yahoo.co.jp/articles/18ba9b1abcce1cbd8c8b13e957fa1b0992f4c071',
 'https://news.yahoo.co.jp/articles/39d352b10cb8c2ab6a1bccaa2f579aecf7db1a26',
 'https://news.yahoo.co.jp/articles/22130ac69e6cd77092e32bf370904802eccd3e92']